In [ ]:
import numpy as np
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
import polars as pl
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce
from typing import List
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import os


In [ ]:
# Reducir memoria automáticamente
def optimizar_memoria(df):
    for col in df.select_dtypes(include=['int64', 'int32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [ ]:
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')

In [ ]:
# Eliminar de df_agrupado las filas donde la columna A_PREDECIR sea 'N'
df_full = df_full[df_full['A_PREDECIR'] != 'N']
df_full = df_full.drop(columns=['A_PREDECIR'])

In [ ]:
df_agrupado = (
    df_full.groupby([
        'ORDINAL','PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE',
        'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'PRODUCT_ID'
    ], as_index=False)[['TN', 'CLASE']].sum()
)

In [ ]:
del df_full
gc.collect()

In [ ]:
# Mostrar las filas de df_agrupado ordernadas por TN de menor a mayor
df_agrupado = df_agrupado.sort_values(by='TN', ascending=True)
# Mostrar las primeras 10 filas de df_agrupado
print(df_agrupado.head(10))

In [ ]:
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
# Calcular los días del mes usando las columnas ANIO y MES

# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si ANIO==2019 y MES en [6, 8], y 0 en caso contrario
df_agrupado['MES_PROBLEMATICO'] = np.where(
       (df_agrupado['ANIO'] == 2019) & (df_agrupado['MES'].isin([6, 8])),
       1, 0
)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from joblib import Parallel, delayed

ROLL_BASE = 12
CAIDA_UMBRAL = -100

def add_global_features(group):
    group = group.sort_values('PERIODO').copy()
    group['FECHA'] = pd.to_datetime(group['PERIODO'].astype(str), format='%Y%m')
    group['MES'] = group['FECHA'].dt.month
    group['IS_FEBRERO'] = (group['MES'] == 2).astype(int)
    group['ESTOY_PREDICIENDO_FEBRERO'] = (group['MES'] == 12).astype(int)

    group['CAIDA_ABRUPTA'] = (group['TN'].diff(1) < CAIDA_UMBRAL).astype(int)
    
    # Tomar solo los febreros históricos
    febreros = group[group['MES'] == 2][['PERIODO', 'TN']].sort_values('PERIODO').reset_index(drop=True)
    # Calcular el rolling promedio (sin el actual, siempre pasado)
    febreros['PROM_ULT_3_FEBREROS'] = febreros['TN'].shift(1).rolling(window=3, min_periods=1).mean()
    
    # Hacer merge con el original: para cada PERIODO, ¿cuál fue el último promedio disponible?
    # Vamos a usar merge_asof para asignar el promedio más reciente antes o igual al periodo actual
    group = group.sort_values('PERIODO').reset_index(drop=True)
    group = pd.merge_asof(
        group, 
        febreros[['PERIODO', 'PROM_ULT_3_FEBREROS']],
        by=None, left_on='PERIODO', right_on='PERIODO', 
        direction='backward', 
        allow_exact_matches=False
    )
    # Si no hay historial, pone cero (puede cambiar a np.nan si preferís)
    group['PROM_ULT_3_FEBREROS'] = group['PROM_ULT_3_FEBREROS'].fillna(0)
    group['DIF_TN_VS_FEBREROS_ULT_3'] = group['TN'] - group['PROM_ULT_3_FEBREROS']



    # Max histórico HASTA el momento (expanding + shift)
    group['TN_MAX_HISTORICO'] = group['TN'].shift(1).expanding().max().fillna(0)
    group['TN_DIST_A_MAX_HIST'] = group['TN_MAX_HISTORICO'] - group['TN']
    group['TN_RATIO_VS_MAX_HIST'] = (group['TN'] / (group['TN_MAX_HISTORICO'] + 1e-8)).clip(lower=0, upper=5)


    return group


def calcular_pendientes_grupo(group, periodos_list):
    group = add_global_features(group)
    group = group.sort_values(by='PERIODO').copy()
    n = len(group)
    y_values = group['TN'].values
    for cant in periodos_list:
        means = np.zeros(n)
        pendientes = np.zeros(n)
        ewmas = np.zeros(n)
        medians = np.zeros(n)
        minimo = np.zeros(n)
        maximo = np.zeros(n)
        stds = np.zeros(n)
        skews = np.zeros(n)
        kurts = np.zeros(n)
        growths = np.zeros(n)
        iqrs = np.zeros(n)
        sums = np.zeros(n)
        count_pos = np.zeros(n)
        pct_zero = np.zeros(n)
        last = np.zeros(n)
        last_diff = np.zeros(n)

        coef_var = np.zeros(n)
        maxmin_ratio = np.zeros(n)
        rango = np.zeros(n)
        rango_rel = np.zeros(n)
        last_vs_median = np.zeros(n)
        cambio_ventana = np.zeros(n)
        zeros_end = np.zeros(n)
        last_pct_sum = np.zeros(n)
        pct_90 = np.zeros(n)
        pct_10 = np.zeros(n)
        pct_width = np.zeros(n)

        # --- Comparaciones TN vs rolling stats ---
        tn_minus_mean = np.zeros(n)
        tn_minus_median = np.zeros(n)
        tn_minus_ewma = np.zeros(n)
        tn_over_mean = np.zeros(n)
        tn_over_median = np.zeros(n)
        tn_over_ewma = np.zeros(n)

        x = np.arange(cant)
        for i in range(n):
            # Solo procesar si la ventana está completa
            if i >= cant - 1:
                y = y_values[i - (cant - 1): i + 1]
                means[i] = np.mean(y)
                try:
                    pendientes[i] = np.polyfit(x, y, 1)[0]
                except:
                    pendientes[i] = 0
                ewmas[i] = pd.Series(y).ewm(span=cant, adjust=False).mean().iloc[-1]
                medians[i] = np.median(y)
                minimo[i] = np.min(y)
                maximo[i] = np.max(y)
                std_y = np.std(y)
                mean_y = means[i]
                stds[i] = std_y
                skews[i] = skew(y) if std_y > 0 else 0
                kurts[i] = kurtosis(y) if std_y > 0 else 0
                growths[i] = (y[-1] - y[0]) / (y[0] + 1e-8) if y[0] != 0 else 0
                iqrs[i] = np.percentile(y, 75) - np.percentile(y, 25)
                sums[i] = np.sum(y)
                count_pos[i] = np.sum(y > 0)
                pct_zero[i] = np.mean(y == 0)
                last[i] = y[-1]
                last_diff[i] = y[-1] - y[-2] if cant > 1 else 0

                coef_var[i] = std_y / (mean_y + 1e-8) if mean_y != 0 else 0
                maxmin_ratio[i] = maximo[i] / (minimo[i] + 1e-8) if minimo[i] != 0 else 0
                rango[i] = maximo[i] - minimo[i]
                rango_rel[i] = rango[i] / (minimo[i] + 1e-8) if minimo[i] != 0 else 0
                last_vs_median[i] = y[-1] - np.median(y)
                # Cambio de ventana anterior a actual
                if i >= 2 * cant - 1:
                    y_prev = y_values[i - (2 * cant - 1):i - cant + 1]
                    mean_prev = np.mean(y_prev)
                    cambio_ventana[i] = (mean_y - mean_prev) / (mean_prev + 1e-8) if mean_prev != 0 else 0
                zeros_end[i] = np.argmax(y[::-1] != 0)
                last_pct_sum[i] = y[-1] / (np.sum(y) + 1e-8) if np.sum(y) != 0 else 0
                pct_90[i] = np.percentile(y, 90)
                pct_10[i] = np.percentile(y, 10)
                pct_width[i] = pct_90[i] - pct_10[i]

                # Comparaciones TN vs rolling stats
                tn_actual = y_values[i]
                tn_minus_mean[i] = tn_actual - means[i]
                tn_minus_median[i] = tn_actual - medians[i]
                tn_minus_ewma[i] = tn_actual - ewmas[i]
                tn_over_mean[i] = tn_actual / (means[i] + 1e-8) if means[i] != 0 else 0
                tn_over_median[i] = tn_actual / (medians[i] + 1e-8) if medians[i] != 0 else 0
                tn_over_ewma[i] = tn_actual / (ewmas[i] + 1e-8) if ewmas[i] != 0 else 0
            # Si la ventana no está completa, todo queda en cero (ya inicializado)

        group[f'TN_MEAN_{str(cant).zfill(2)}'] = means
        group[f'PENDIENTE_TENDENCIA_{cant}'] = pendientes
        group[f'TN_EWMA_{str(cant).zfill(2)}'] = ewmas
        group[f'TN_MEDIAN_{str(cant).zfill(2)}'] = medians
        group[f'TN_MIN_{str(cant).zfill(2)}'] = minimo
        group[f'TN_MAX_{str(cant).zfill(2)}'] = maximo
        group[f'TN_STD_{str(cant).zfill(2)}'] = stds
        group[f'TN_SKEW_{str(cant).zfill(2)}'] = skews
        group[f'TN_KURT_{str(cant).zfill(2)}'] = kurts
        group[f'TN_GROWTH_{str(cant).zfill(2)}'] = growths
        group[f'TN_IQR_{str(cant).zfill(2)}'] = iqrs
        group[f'TN_SUM_{str(cant).zfill(2)}'] = sums
        group[f'TN_COUNT_POS_{str(cant).zfill(2)}'] = count_pos
        group[f'TN_PCT_ZERO_{str(cant).zfill(2)}'] = pct_zero
        group[f'TN_LAST_{str(cant).zfill(2)}'] = last
        group[f'TN_LAST_DIFF_{str(cant).zfill(2)}'] = last_diff

        group[f'TN_COEF_VAR_{cant}'] = coef_var
        group[f'TN_MAXMIN_RATIO_{cant}'] = maxmin_ratio
        group[f'TN_RANGO_{cant}'] = rango
        group[f'TN_RANGO_REL_{cant}'] = rango_rel
        group[f'TN_LAST_VS_MEDIAN_{cant}'] = last_vs_median
        group[f'TN_CHANGE_PREV_WINDOW_{cant}'] = cambio_ventana
        group[f'TN_ZEROS_END_{cant}'] = zeros_end
        group[f'TN_LAST_PCT_SUM_{cant}'] = last_pct_sum
        group[f'TN_PCT90_{cant}'] = pct_90
        group[f'TN_PCT10_{cant}'] = pct_10
        group[f'TN_PCT_WIDTH_{cant}'] = pct_width

        # Comparaciones TN vs rolling stats
        group[f'TN_MINUS_MEAN_{str(cant).zfill(2)}'] = tn_minus_mean
        group[f'TN_MINUS_MEDIAN_{str(cant).zfill(2)}'] = tn_minus_median
        group[f'TN_MINUS_EWMA_{str(cant).zfill(2)}'] = tn_minus_ewma
        group[f'TN_OVER_MEAN_{str(cant).zfill(2)}'] = tn_over_mean
        group[f'TN_OVER_MEDIAN_{str(cant).zfill(2)}'] = tn_over_median
        group[f'TN_OVER_EWMA_{str(cant).zfill(2)}'] = tn_over_ewma

    return group

In [ ]:
def calcular_pendientes_parallel(df, periodos_list=[3, 6, 9, 12, 18], n_jobs=28):
    grupos = [group for _, group in df.groupby('PRODUCT_ID')]
    resultados = Parallel(n_jobs=n_jobs, backend='loky', verbose=10)(
        delayed(calcular_pendientes_grupo)(group, periodos_list) for group in grupos
    )
    df_final = pd.concat(resultados, ignore_index=True)
    df_final.replace([np.inf, -np.inf], 0, inplace=True)
    df_final.fillna(0, inplace=True)
    return df_final

df_agrupado = calcular_pendientes_parallel(df_agrupado, periodos_list=[3, 6, 9, 12, 18], n_jobs=28)


In [ ]:
# Imprimir las columnas PRODUCT_ID,PERIODO,TN y las que contienen PENDIENTE de df_agrupado que muestre 100 columnas por pantalla
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.precision', 2)

# Mostrar solo las columnas PRODUCT_ID, PERIODO, TN y las que contienen 'PENDIENTE'
cols = [col for col in df_agrupado.columns if col in ['PRODUCT_ID', 'PERIODO', 'TN'] or 'PENDIENTE' in col]
# Mostrar las últimas 10 filas de df_agrupado de PRODUCT_ID== 20001
display(df_agrupado[cols][df_agrupado['PRODUCT_ID'] == 20001])




In [ ]:
import pandas as pd

lags = [3, 6, 9, 12, 18]

# Definí la columna de agrupamiento, por ejemplo:
group_cols = ['PRODUCT_ID']  # Cambia esto si tu agrupamiento es por otra cosa

# Ordenar el DataFrame por las claves necesarias
df_agrupado = df_agrupado.sort_values(group_cols + ['ORDINAL'], ascending=[True, False])

for lag in lags:
    lag_col = f'TN_LAG_{lag:02d}'
    delta_col = f'TN_DELTA_{lag:02d}'
    # Calcular el lag
    df_agrupado[lag_col] = (
        df_agrupado.groupby(group_cols)['TN']
        .shift(-lag)  # Sentido descendente en ORDINAL
        .fillna(0)
    )
    # Calcular el delta
    df_agrupado[delta_col] = df_agrupado['TN'] - df_agrupado[lag_col]

# Si querés, podés ver las primeras filas
print(df_agrupado[[f'TN_LAG_{l:02d}' for l in lags] + [f'TN_DELTA_{l:02d}' for l in lags]].head())


In [ ]:
import numpy as np
import pandas as pd

windows = [3, 6, 9, 12, 18]
group_cols = ['PRODUCT_ID']  # agregá más claves si corresponde

for w in windows:
    # CRECIMIENTO PERCENTUAL (respecto al lag de la ventana)
    lag_col = f'TN_LAG_{w:02d}'
    growth_col = f'TN_GROWTH_{w:02d}'
    if lag_col in df_agrupado.columns:
        df_agrupado[growth_col] = (df_agrupado['TN'] - df_agrupado[lag_col]) / (df_agrupado[lag_col].replace(0, np.nan))
        df_agrupado[growth_col] = df_agrupado[growth_col].fillna(0)
    
    # CANTIDAD DE CEROS EN LA VENTANA
    count_zero_col = f'TN_COUNT_ZERO_{w:02d}'
    df_agrupado[count_zero_col] = (
        df_agrupado
        .groupby(group_cols)['TN']
        .rolling(window=w, min_periods=1)
        .apply(lambda x: np.sum(x == 0), raw=True)
        .reset_index(level=0, drop=True)
    )
    
    # NÚMERO DE CAMBIOS DE SIGNO EN LA VENTANA
    sign_change_col = f'TN_SIGN_CHANGES_{w:02d}'
    df_agrupado[sign_change_col] = (
        df_agrupado
        .groupby(group_cols)['TN']
        .rolling(window=w, min_periods=2)
        .apply(lambda x: np.sum(np.diff(np.sign(x)) != 0), raw=True)
        .reset_index(level=0, drop=True)
    )
    
    # COEFICIENTE DE VARIACIÓN EN LA VENTANA
    coef_var_col = f'TN_COEF_VAR_{w:02d}'
    mean_col = f'TN_MEAN_{w:02d}'
    std_col = f'TN_STD_{w:02d}'
    # Si ya tenés mean y std, sólo calculás la división, si no, calculás aquí
    if mean_col in df_agrupado.columns and std_col in df_agrupado.columns:
        df_agrupado[coef_var_col] = df_agrupado[std_col] / (df_agrupado[mean_col].replace(0, np.nan))
        df_agrupado[coef_var_col] = df_agrupado[coef_var_col].fillna(0)
    else:
        df_agrupado[mean_col] = (
            df_agrupado.groupby(group_cols)['TN']
            .rolling(window=w, min_periods=1).mean()
            .reset_index(level=0, drop=True)
        )
        df_agrupado[std_col] = (
            df_agrupado.groupby(group_cols)['TN']
            .rolling(window=w, min_periods=1).std().fillna(0)
            .reset_index(level=0, drop=True)
        )
        df_agrupado[coef_var_col] = df_agrupado[std_col] / (df_agrupado[mean_col].replace(0, np.nan))
        df_agrupado[coef_var_col] = df_agrupado[coef_var_col].fillna(0)
    
    # PROPORCIÓN DE OUTLIERS (> 2 * std en la ventana)
    outlier_col = f'TN_OUTLIER_PROP_{w:02d}'
    def outlier_prop(x):
        m = np.nanmean(x)
        s = np.nanstd(x)
        if s == 0:
            return 0
        return np.mean(np.abs(x - m) > 2 * s)
    df_agrupado[outlier_col] = (
        df_agrupado
        .groupby(group_cols)['TN']
        .rolling(window=w, min_periods=2)
        .apply(outlier_prop, raw=True)
        .reset_index(level=0, drop=True)
    )

    # PROPORCIÓN DE VENTAS MAYORES A CERO
    pct_pos_col = f'TN_PCT_POS_{w:02d}'
    df_agrupado[pct_pos_col] = (
        df_agrupado
        .groupby(group_cols)['TN']
        .rolling(window=w, min_periods=1)
        .apply(lambda x: np.mean(x > 0), raw=True)
        .reset_index(level=0, drop=True)
    )

# Si querés ver un resumen de las nuevas columnas:
cols_new = [
    f'TN_GROWTH_{w:02d}' for w in windows
] + [
    f'TN_COUNT_ZERO_{w:02d}' for w in windows
] + [
    f'TN_SIGN_CHANGES_{w:02d}' for w in windows
] + [
    f'TN_COEF_VAR_{w:02d}' for w in windows
] + [
    f'TN_OUTLIER_PROP_{w:02d}' for w in windows
] + [
    f'TN_PCT_POS_{w:02d}' for w in windows
]
print(df_agrupado[cols_new].head())

In [ ]:
print(df_agrupado.columns.tolist())

In [ ]:
# Hacer un EDA de la columna  CLASE en  df_agrupado
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Estadísticas descriptivas ---
print("Estadísticas básicas de CLASE:")
print(df_agrupado['CLASE'].describe())

print("\nPrimeros valores únicos (si son pocos):", df_agrupado['CLASE'].unique()[:10])
print("Cantidad de valores únicos:", df_agrupado['CLASE'].nunique())

print("\nCantidad de nulos:", df_agrupado['CLASE'].isna().sum())

# --- Histograma ---
plt.figure(figsize=(8,4))
sns.histplot(df_agrupado['CLASE'], bins=30, kde=True)
plt.title("Histograma de CLASE")
plt.xlabel("CLASE")
plt.ylabel("Frecuencia")
plt.show()

# --- Boxplot para ver outliers ---
plt.figure(figsize=(8,2))
sns.boxplot(x=df_agrupado['CLASE'])
plt.title("Boxplot de CLASE")
plt.show()

# --- Distribución log-transformada (útil si hay muchos valores chicos y pocos grandes) ---
plt.figure(figsize=(8,4))
sns.histplot(np.log1p(df_agrupado['CLASE']), bins=30, kde=True)
plt.title("Histograma de log(1+CLASE)")
plt.xlabel("log(1+CLASE)")
plt.ylabel("Frecuencia")
plt.show()

# --- Percentiles y cuantiles ---
print("\nPercentiles de CLASE:")
print(df_agrupado['CLASE'].quantile([0, 0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99, 1]))

# --- Outliers extremos (más allá de 3 desviaciones estándar) ---
media = df_agrupado['CLASE'].mean()
std = df_agrupado['CLASE'].std()
outliers = df_agrupado[(df_agrupado['CLASE'] > media + 3*std) | (df_agrupado['CLASE'] < media - 3*std)]
print(f"\nCantidad de outliers (>3 desvíos): {len(outliers)}")

# --- Relación con otras variables (ejemplo: PERIODO) ---
if 'PERIODO' in df_agrupado.columns:
    plt.figure(figsize=(10,4))
    sns.lineplot(x='PERIODO', y='CLASE', data=df_agrupado, ci=None)
    plt.title("CLASE a lo largo del tiempo (PERIODO)")
    plt.show()




In [ ]:
df_agrupado['CLASE_LOG1P'] = np.log1p(df_agrupado['CLASE'])
# Y para invertir:
#df_agrupado['CLASE'] = np.expm1(df_full['CLASE_LOG1P'])

# Eliminar la columna CLASE 
df_agrupado = df_agrupado.drop(columns=['CLASE'])


In [ ]:
# Eliminar las columnas CLASE  para evitar confusiones
df_agrupado.drop(columns=['CLASE'], inplace=True, errors='ignore')

In [ ]:
print(df_agrupado.columns.tolist())

In [ ]:
# Guardar el DataFrame en parquet
df_agrupado.to_parquet('./data/product_interm_LGBM.parquet', index=False, engine='fastparquet')

In [ ]:
df_agrupado = pd.read_parquet('./data/product_interm_LGBM.parquet', index=False, engine='fastparquet')

In [ ]:
# Excluir la variable de clase
cols_to_check = df_agrupado.columns.difference(['CLASE_LOG1P'])

# Calcular cantidad de NaNs por columna
nan_columns = df_agrupado[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

In [ ]:
cols_to_check = df_agrupado.columns.difference(['CLASE_LOG1P'])

# Reemplaza NaN e infinitos por 0 solo en las columnas seleccionadas
df_agrupado[cols_to_check] = df_agrupado[cols_to_check].replace([np.nan, np.inf, -np.inf], 0)

# Excluir la variable de clase
cols_to_check = df_agrupado.columns.difference(['CLASE_LOG1P'])

# Calcular cantidad de NaNs por columna
nan_columns = df_agrupado[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

In [ ]:
print(f"📊 DataFrame final con {df_agrupado.shape[0]:,} filas y {df_agrupado.shape[1]} columnas:")

In [ ]:
# Copia de seguridad del DataFrame
df = df_agrupado.copy()

# === Binning (en deciles) ===
df['PRODUCT_RANK_BIN'] = pd.qcut(df['PRODUCT_ID'], q=10, labels=False)
df['PRODUCT_RANK_BIN'] = df['PRODUCT_RANK_BIN'].astype('category')

# Reemplazar en el DataFrame principal
df_agrupado = df


In [ ]:
print(df_agrupado.columns.tolist())

In [ ]:
# Eliminar de df_agrupado la columna FECHA
df_agrupado = df_agrupado.drop(columns=['FECHA'], errors='ignore')

In [ ]:
cat_cols = ['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE','MES_PROBLEMATICO',
            'PRODUCT_RANK_BIN','IS_FEBRERO', 'ESTOY_PREDICIENDO_FEBRERO','CAIDA_ABRUPTA']

for col in cat_cols:
    if col in df_agrupado.columns:
        df_agrupado[col] = df_agrupado[col].astype('category')

In [ ]:
df_agrupado.to_parquet('./data/product_interm_LGBM.parquet', index=False, engine='fastparquet')